In [1]:
import pandas as pd
import numpy as np

In [2]:
def get_media_type(annot, media='audio'):
    les_medias = set(['audio','video','texte'])
    les_medias.difference(set([media]))
    
    filtres = {}
    filtres['isGraded'] = annot['isGraded'] == 1
    filtres[media]      = annot[media] == 1
    f = pd.DataFrame(filtres).apply(sum, axis=1) == len(filtres)
    df_media = annot.loc[f,:].copy()
    
    for m in les_medias.difference(set([media])):
        filtres[m] = annot[m] == 0
    f = pd.DataFrame(filtres).apply(sum, axis=1) == len(filtres)
    df_media_only = annot.loc[f,:].copy()
    
    if len(df_media) > 0:
        df_media['moyenne']      = df_media[noteurs].apply(np.nanmean, axis=1)
        print(media," : ",      len(set(df_media['doc'])), sep="")
    
    if len(df_media_only) > 0:
        df_media_only['moyenne'] = df_media_only[noteurs].apply(np.nanmean, axis=1)
        print(media,"_only : ", len(set(df_media_only['doc'])), sep="")
        
        df_media_only[['doc','moyenne']].groupby(['doc']).agg(['count','mean']).head()
    
    return df_media, df_media_only

#### Ouverture du csv des annotations, changement des colonnes

In [3]:
samplerate = 16000

directory = './challenge-m2-sid/'
dir_audio = directory + 'corpus/audio/'
dir_texte = directory + 'corpus/text/'
dir_video = directory + 'corpus/video/'

audio_extension = "_mono.wav"
texte_extension = ".xml"
video_extension = ".m4v"

annot = pd.read_csv(directory+'annotations_challenge_sid.csv', sep="\t", index_col=0, header=0).replace(-1,np.nan)
noteurs = annot.columns[1:]

annot['doc']      = ["_".join(e.split("_")[0:2]) for e in annot["code_doc"]]
annot['audio']    = [int(e.split("_")[2][0])     for e in annot["code_doc"]]
annot['video']    = [int(e.split("_")[2][1])     for e in annot["code_doc"]]
annot['texte']    = [int(e.split("_")[2][2])     for e in annot["code_doc"]]
annot['isGraded'] = [int(e.split("_")[3])        for e in annot["code_doc"]]

annot = annot.drop("code_doc", axis=1)
annot.head(6)

,il08_09,vg04_05,fd03_04,la09_10,cg13_14,mb00_12,doc,audio,video,texte,isGraded
1,NaN,NaN,NaN,NaN,NaN,NaN,119_8,1,0,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,119_8,1,1,0,0
3,NaN,NaN,NaN,NaN,NaN,NaN,119_8,1,1,1,0
4,NaN,NaN,NaN,NaN,NaN,NaN,119_8,1,0,1,0
5,NaN,NaN,NaN,NaN,NaN,NaN,119_8,0,0,1,0
6,NaN,NaN,NaN,NaN,NaN,76.0,57_6,1,0,0,1


#### Selection des audio notés et calcul de la moyenne

In [4]:
audio, audio_only = get_media_type(annot, media='audio')

audio : 55
audio_only : 33


In [5]:
video, video_only = get_media_type(annot, media='video')

video : 44


In [6]:
texte, texte_only = get_media_type(annot, media='texte')

texte : 55
texte_only : 33


texte_only = annot[(annot['isGraded'] == 1) &
                   (annot['audio']    == 0) &
                   (annot['video']    == 0) &
                   (annot['texte']    == 1)
                  ].copy()

texte = annot[(annot['isGraded'] == 1) & 
              (annot['texte']    == 1)
             ].copy()

texte['moyenne']      = texte[noteurs].apply(np.nanmean, axis=1)
texte_only['moyenne'] = texte_only[noteurs].apply(np.nanmean, axis=1)

print("texte :",      len(set(texte['doc'])))
print("texte_only :", len(set(texte_only['doc'])))

texte_only[['doc','moyenne']].groupby(['doc']).agg(['count','mean']).head(7)